In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sys, importlib
from scripts.classes import Label, Sample
import scripts.util as util
importlib.reload(sys.modules['scripts.util'])



print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Data loading step

In [2]:
TRAIN_DATA = "../data/EuroSatData/train.csv"
TEST_DATA = "../data/EuroSatData/test.csv"
VALIDATION_DATA = "../data/EuroSatData/validation.csv"

# Load data into pandas dataframe from csv files
train_df = pd.read_csv(TRAIN_DATA)
train_df["Filename"] = "../data/EuroSatData/" + train_df["Filename"].astype(str)
test_df = pd.read_csv(TEST_DATA)
test_df["Filename"] = "../data/EuroSatData/" + test_df["Filename"].astype(str)
validation_df = pd.read_csv(VALIDATION_DATA)
validation_df["Filename"] = "../data/EuroSatData/" + validation_df["Filename"].astype(str)

In [3]:
# Load image data into new column called "Image"
train_df["Image"] = train_df["Filename"].apply(util.read_img)
test_df["Image"] = test_df["Filename"].apply(util.read_img)
validation_df["Image"] = validation_df["Filename"].apply(util.read_img)

In [4]:
# Prepare the x and y of each data set
x_train, y_train = util.pd_series_to_tensor(train_df["Image"]), train_df["Label"].values
x_test, y_test = util.pd_series_to_tensor(test_df["Image"]), test_df["Label"].values
x_validation, y_validation = util.pd_series_to_tensor(validation_df["Image"]), validation_df["Label"].values

# Model set up and training

In [5]:
pretrained_vgg16 = tf.keras.applications.vgg16.VGG16(include_top=False, input_shape=(64, 64, 3))

In [6]:
pretrained_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [7]:
model = tf.keras.Sequential()

In [8]:
# Ignore the last layer, which is the dense layers
for layer in pretrained_vgg16.layers:
  layer.trainable = False
  model.add(layer)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 16, 16, 256)       2

In [10]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=10))
model.add(tf.keras.layers.Activation("softmax"))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 16, 16, 256)       2

In [12]:
# lr_sched = tf.keras.optimizers.schedules.ExponentialDecay(
#   initial_learning_rate=0.001,
#   decay_steps=10000,
#   decay_rate=0.9
# )

model.compile(
  optimizer=tf.optimizers.Adam(learning_rate=0.001),
  loss=tf.losses.SparseCategoricalCrossentropy(),
  metrics=["accuracy"]
)

In [13]:
# NOTE: Possible batch_size configuration
train_history = model.fit(x=x_train, y=y_train, validation_data=(x_validation, y_validation), batch_size=16, epochs=20, verbose=2)

Epoch 1/20
1182/1182 - 85s - loss: 0.7354 - accuracy: 0.7570 - val_loss: 0.5519 - val_accuracy: 0.8148 - 85s/epoch - 72ms/step
Epoch 2/20
1182/1182 - 65s - loss: 0.4862 - accuracy: 0.8356 - val_loss: 0.5075 - val_accuracy: 0.8315 - 65s/epoch - 55ms/step
Epoch 3/20
1182/1182 - 80s - loss: 0.4213 - accuracy: 0.8593 - val_loss: 0.4459 - val_accuracy: 0.8494 - 80s/epoch - 68ms/step
Epoch 4/20
1182/1182 - 72s - loss: 0.3830 - accuracy: 0.8721 - val_loss: 0.4331 - val_accuracy: 0.8561 - 72s/epoch - 61ms/step
Epoch 5/20
1182/1182 - 57s - loss: 0.3537 - accuracy: 0.8808 - val_loss: 0.4137 - val_accuracy: 0.8570 - 57s/epoch - 48ms/step
Epoch 6/20
1182/1182 - 59s - loss: 0.3351 - accuracy: 0.8884 - val_loss: 0.4044 - val_accuracy: 0.8641 - 59s/epoch - 50ms/step
Epoch 7/20
1182/1182 - 38s - loss: 0.3174 - accuracy: 0.8922 - val_loss: 0.4112 - val_accuracy: 0.8624 - 38s/epoch - 32ms/step
Epoch 8/20
1182/1182 - 49s - loss: 0.3029 - accuracy: 0.8978 - val_loss: 0.4022 - val_accuracy: 0.8635 - 49s/ep

In [ ]:
# Show train losses and accuracy here

In [14]:
# NOTE: Possible batch_size configuration
prediction = model.predict(x=x_test, verbose=2)
y_prediction = np.argmax(prediction, axis=1)

85/85 - 101s - 101s/epoch - 1s/step


In [15]:
correct = y_test == y_prediction
accuracy = np.sum(correct)/2700
accuracy

0.8818518518518519